In [19]:
import smtplib
from email.mime.text import MIMEText
import time

# Email configuration
SMTP_SERVER = "smtp.gmail.com"  # For Gmail, change if using Outlook or another service
SMTP_PORT = 465  # SSL port (or use 587 for TLS)
SENDER_EMAIL = "abhaymathur1000@gmail.com"  # Replace with your email
SENDER_PASSWORD = "blom axpb quot zcdv"  # Replace with your email password
RECEIVER_EMAIL = "abhaymathur1000@gmail.com"  # Your email (or another recipient)

def send_email(subject, body=""):
    """Sends an email notification"""
    msg = MIMEText(body)
    msg["Subject"] = subject
    msg["From"] = SENDER_EMAIL
    msg["To"] = RECEIVER_EMAIL

    try:
        with smtplib.SMTP_SSL(SMTP_SERVER, SMTP_PORT) as server:
            server.login(SENDER_EMAIL, SENDER_PASSWORD)
            server.sendmail(SENDER_EMAIL, RECEIVER_EMAIL, msg.as_string())
        print("Email sent successfully!")
    except Exception as e:
        print(f"Error sending email: {e}")



# Notify when the script completes
# send_email("✅ Script Completed", "Your Python script has finished running.")

start_time = time.time()

def get_time_from_start(start_time):
    elapsed_time = time.time() - start_time
    hours, rem = divmod(elapsed_time, 3600)
    minutes, seconds = divmod(rem, 60)
    return f"{int(hours):02}:{int(minutes):02}:{seconds:05.2f}"


# Imports

In [ ]:
# Import Libraries
import os
import json
import sys
import pandas as pd
import kagglehub
import time
import datetime
import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler, random_split
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.preprocessing import MultiLabelBinarizer
import transformers
from transformers import BertForSequenceClassification, AdamW, BertConfig, BertTokenizer, get_linear_schedule_with_warmup, BertModel
from IPython.display import display, clear_output
import dask.dataframe as dd
import numpy as np
from scipy.stats import gaussian_kde
import matplotlib.pyplot as plt
import seaborn as sns

import traceback
import sys

import nltk
from nltk.tag import pos_tag
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
nltk.download('universal_tagset')
nltk.download('stopwords')
try:
    nltk.download('averaged_perceptron_tagger')
except:
    pass
try:
    nltk.download('averaged_perceptron_tagger_eng')
except:
    pass


from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
import spacy

os.system('./venv/bin/python -m spacy download en_core_web_sm')
import vaderSentiment
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
try:
    nltk.download('punkt')
except:
    pass
try:
    nltk.download('punkt_tab')
except:
    pass

from nltk.tokenize import sent_tokenize


[nltk_data] Downloading package universal_tagset to
[nltk_data]     /home/abhay/nltk_data...
[nltk_data]   Package universal_tagset is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/abhay/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     /home/abhay/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger_eng is already up-to-
[nltk_data]       date!


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 25.2 MB/s eta 0:00:0000:0100:01
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


[nltk_data] Downloading package punkt to /home/abhay/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /home/abhay/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


In [21]:
if os.path.exists("/kaggle"):
    path = "/kaggle/input/yelp-dataset"
elif os.path.exists('/home/abhaydesktop/.cache/kagglehub/datasets/yelp-dataset/yelp-dataset/versions/4'):
    path = '/home/abhaydesktop/.cache/kagglehub/datasets/yelp-dataset/yelp-dataset/versions/4'
else:
    path = kagglehub.dataset_download("yelp-dataset/yelp-dataset")
print("Path to dataset files:", path)



def load_data(path, filename, chunk_size=10000):
    chunks = pd.read_json(os.path.join(path, filename), lines=True, chunksize=chunk_size)

    df_list = []

    for chunk in chunks:
        df_list.append(chunk)


    return pd.concat(df_list, ignore_index=True)



Path to dataset files: /home/abhay/.cache/kagglehub/datasets/yelp-dataset/yelp-dataset/versions/4


# Model

In [ ]:
class BertClassifier(nn.Module):
	def __init__(self, data_path, tensor_path=None, model_name="bert-base-uncased", batch_size=32, epochs=1, n_part=0):
		"""
		Initializes the BertClassifier.

		Args:
			data_path (str): Path to the directory containing the Parquet files.
			model_name (str, optional): Name of the BERT model to use. Defaults to "bert-base-uncased".
			num_labels (int, optional): Number of output labels. If None, it will be inferred from the data. Defaults to None.
			batch_size (int, optional): Batch size for training and evaluation. Defaults to 32.
			epochs (int, optional): Number of training epochs. Defaults to 1.
		"""
		super(BertClassifier, self).__init__()

		self.data_path = data_path
		self.model_name = model_name
		self.batch_size = batch_size
		self.epochs = epochs
		self.tokenizer_max_length = 512
		self.device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

		if tensor_path is None:
			print("Loading in the data...")

			# Load data and preprocess
			if n_part > 0:
				file_count = len([f for f in os.listdir(self.data_path) if os.path.isfile(os.path.join(self.data_path, f))])

				for i in range(min(n_part, file_count)):
					read_df = dd.read_parquet(os.path.join(self.data_path, f"output_part_{i}.parquet"), engine="pyarrow")
					self.df_dask = read_df if i == 0 else dd.concat([self.df_dask, read_df])
			else:
				self.df_dask = dd.read_parquet(self.data_path, engine="pyarrow")
			
			self.all_df = self.df_dask.compute()

			print("Loaded in the data")
			
			# Set up the data specific stuff
			self.businesses_df = load_data(path, "yelp_academic_dataset_business.json")

			# Sample 20 reviews per business and concate
			def get_adj(text):
					tokenized = pos_tag(word_tokenize(text), tagset='universal')
					# print(tokenized)
					adjs = [word for word, pos in tokenized if pos == 'ADJ']
					return ' '.join(adjs)

			def sentiment_summary(text, fraction=0.3):
				"""
				Summarizes text while preserving sentiment.
				
				:param text: The input text to summarize
				:param fraction: The percentage of original text to keep (0.0 to 1.0)
				:return: A summarized version of the text
				"""
				sentences = sent_tokenize(text)
				total_sentences = len(sentences)
				
				# Ensure at least 1 sentence is returned
				num_sentences = max(1, int(total_sentences * fraction))

				vectorizer = TfidfVectorizer(stop_words="english")
				tfidf_matrix = vectorizer.fit_transform(sentences)
				
				sentence_scores = np.array(tfidf_matrix.sum(axis=1)).flatten()
				
				# Get sentiment scores
				analyzer = SentimentIntensityAnalyzer()
				sentiment_scores = [abs(analyzer.polarity_scores(s)['compound']) for s in sentences]
				
				# Combine importance & sentiment (weighted sum)
				combined_scores = sentence_scores * np.array(sentiment_scores)
				
				# Pick top sentences
				top_sentence_indices = combined_scores.argsort()[-num_sentences:][::-1]
				summary = [sentences[i] for i in sorted(top_sentence_indices)]
				
				return " ".join(summary)
			
			# Print the length of all the texts
			print("Length of all the texts before anything: ", self.all_df['text'].apply(lambda x: len(x)).describe())

			try:
				frac = 0.3
				self.all_df['text'] = self.all_df['text'].apply(lambda x : sentiment_summary(x, fraction=frac))

				# Print the length of all the texts
				print("Length of all the texts after sentiment summary: ", self.all_df['text'].apply(lambda x: len(x)).describe())
			except Exception as e:
				print("Error in sentiment summary, skipping over")


			self.all_df['text'] = self.all_df['text'].apply(lambda x : get_adj(x))

			# Print the length of all the texts
			print("Length of all the texts after adj only: ", self.all_df['text'].apply(lambda x: len(x)).describe())

			# Cap the text at a certain length
			self.all_df['text'] = self.all_df['text'].apply(lambda x: x[:int(self.tokenizer_max_length / 5)])

			# Print the length of all the texts
			print("Length of all the texts after capping: ", self.all_df['text'].apply(lambda x: len(x)).describe())
 
			def calculate_return_guest_percentage(reviews_business_df):
				df = reviews_business_df  # Ensure it has columns: 'user_id', 'business_id', 'text'
				
				# Count total unique guests per business
				total_guests = df.groupby('business_id')['user_id'].nunique().reset_index()
				total_guests = total_guests.rename(columns={'user_id': 'total_guest_count'})

				# Count the number of reviews per user per business
				repeat_visits = df.groupby(['business_id', 'user_id']).size().reset_index(name='review_count')

				# Filter businesses where a user has more than one review (return guests)
				repeat_visits = repeat_visits[repeat_visits['review_count'] > 1]

				# Count return guests per business
				return_guests = repeat_visits.groupby('business_id')['user_id'].nunique().reset_index()
				return_guests = return_guests.rename(columns={'user_id': 'return_guest_count'})

				# Merge return guests count with total guests count
				result = total_guests.merge(return_guests, on='business_id', how='left').fillna(0)

				# Calculate return guest percentage
				result['return_percentage'] = (result['return_guest_count'] / result['total_guest_count']) * 100
				result['return_percentage'] = result['return_percentage'].fillna(0)  # Replace NaN with 0%

				return result
			
			return_guest_percentage = calculate_return_guest_percentage(self.all_df)


			sampled_reviews = self.all_df.groupby('business_id')['text'].apply(lambda x: ' '.join(x.sample(min(len(x), 20))))
			sampled_reviews_businesses_df = pd.merge(self.businesses_df, sampled_reviews, on='business_id', how='inner')
			self.all_df['date'] = pd.to_datetime(self.all_df['date'])

			last_review_dates = self.all_df.groupby('business_id')['date'].max().reset_index()

			businesses_before_2020 = last_review_dates[last_review_dates['date'] < '2020-01-01']

			sampled_reviews_businesses_not_after_2020 = pd.merge(sampled_reviews_businesses_df, businesses_before_2020, on='business_id', how='inner')
			sampled_reviews_businesses_after_2020 = sampled_reviews_businesses_df[~sampled_reviews_businesses_df['business_id'].isin(businesses_before_2020['business_id'])]

			# add return_guest_percentage to all_df
			sampled_reviews_businesses_not_after_2020 = sampled_reviews_businesses_not_after_2020.merge(return_guest_percentage, on='business_id', how='left')
			sampled_reviews_businesses_after_2020 = sampled_reviews_businesses_after_2020.merge(return_guest_percentage, on='business_id', how='left')

			print("Return Guest Percentage: ", sampled_reviews_businesses_not_after_2020['return_percentage'].describe())

			def return_useful_features(df):
				name_counts = df['name'].value_counts()
				df['franchise_score'] = df['name'].apply(lambda x: np.log(name_counts[x]))
				df['franchise_score'] = df['franchise_score'].fillna(0).astype(float)

				# One hot encode the state and add it to df
				df = pd.concat([df, pd.get_dummies(df['state'])], axis=1)
				print("DF columns: ", df.columns)



				coords = df[['latitude', 'longitude']].values.T

				kde = gaussian_kde(coords)
				df['density_score'] = kde(coords).astype(float)

				# Normalize it
				df['density_score'] = (df['density_score'] - df['density_score'].min()) / (df['density_score'].max() - df['density_score'].min())


				useful_features = ['latitude', 'longitude', 'stars', 'review_count', 'franchise_score', 'density_score', 'return_percentage', 'text']
				df_useful_features = df[useful_features]
				return df_useful_features, df_useful_features.shape[1] - 1, useful_features

			sampled_reviews_businesses_not_after_2020, nn_input_size, useful_features = return_useful_features(sampled_reviews_businesses_not_after_2020)
			self.nn_input_size = nn_input_size
			sampled_reviews_businesses_after_2020, _, _ = return_useful_features(sampled_reviews_businesses_after_2020)

			num_businesses_before_2020 = sampled_reviews_businesses_not_after_2020.shape[0]
			num_businesses_after_2020 = sampled_reviews_businesses_after_2020.shape[0]
			print("Number of businesses before 2020: ", num_businesses_before_2020)
			print("Number of businesses after 2020: ", num_businesses_after_2020)
			print("Percentage of businesses before 2020: ", num_businesses_before_2020 / (num_businesses_before_2020 + num_businesses_after_2020))
			print("Percentage of businesses after 2020: ", num_businesses_after_2020 / (num_businesses_before_2020 + num_businesses_after_2020))

			sampled_reviews_businesses_not_after_2020['output'] = 0
			sampled_reviews_businesses_after_2020['output'] = 1

			# I am taking a stratified sample of the data to better represent the 0s
			num_samples = min(num_businesses_before_2020, num_businesses_after_2020)

			sampled_reviews_businesses_not_after_2020 = sampled_reviews_businesses_not_after_2020.sample(n=num_samples, random_state=42)
			sampled_reviews_businesses_after_2020 = sampled_reviews_businesses_after_2020.sample(n=num_samples, random_state=42)


			self.final_dataset = pd.concat([sampled_reviews_businesses_not_after_2020, sampled_reviews_businesses_after_2020], ignore_index=True)
			print("Total dataset length: ", len(self.final_dataset))


		self.tokenizer = BertTokenizer.from_pretrained(self.model_name, do_lower_case=True)

		if tensor_path is not None and os.path.exists(tensor_path):
			tensors = torch.load(tensor_path)
			self.bert_inputs = tensors["bert_inputs"]
			self.nn_inputs = tensors["nn_inputs"]
			self.nn_input_size = self.nn_inputs.shape[1]
			self.attention_masks = tensors["attention_masks"]
			self.outputs = tensors["outputs"]
		else:
			self.bert_inputs, self.nn_inputs, self.attention_masks, self.outputs = self._preprocess_data(useful_features, tensor_save_path=f'data/tensors/tensors_parts{n_part}.pth')
		self.train_dataloader, self.validation_dataloader, self.test_dataloader = self._create_dataloaders()

		self.model = BertModel.from_pretrained(self.model_name).to(self.device)

		for param in self.model.parameters():
			param.requires_grad = False

		for param in self.model.encoder.layer[-2:].parameters():
			param.requires_grad = True

		self.bert_output_size = self.model.config.hidden_size

		self.nn_hidden_size = 64
		self.nn_output_size = 32
		self.final_hidden_size = 16

		self.nn_layers = nn.Sequential(
			nn.Linear(self.nn_input_size, self.nn_hidden_size),
			nn.ReLU(),
			nn.Linear(self.nn_hidden_size, self.nn_output_size),
			nn.ReLU()
		).to(self.device)

		self.final_layers = nn.Sequential(
			nn.Linear(self.bert_output_size + self.nn_output_size, self.final_hidden_size),
			nn.ReLU(),
			nn.Linear(self.final_hidden_size, 1)
		).to(self.device)

		self.optimizer = AdamW(self.model.parameters(), lr=2e-5, eps=1e-8)
		self.scheduler = self._get_scheduler()
		self.criterion = nn.BCEWithLogitsLoss()

	def _preprocess_data(self, useful_features, tensor_save_path=None):
		"""Preprocesses the data."""
		inputs = self.final_dataset["text"].values
		print("Inputs: ", inputs)
		print("Inputs Shape: ", inputs.shape)
		print("Dataset Useful Features: ", self.final_dataset[useful_features].head(5))

		if "text" in useful_features:
			useful_features.remove("text")

		# Ensure all useful features are numeric and handle missing values
		for feature in useful_features:
			self.final_dataset[feature] = pd.to_numeric(self.final_dataset[feature], errors='coerce').fillna(0)

		# Convert to torch tensor
		nn_inputs = torch.tensor(self.final_dataset[useful_features].values, dtype=torch.float32)
		
		print("NN Inputs: ", nn_inputs)
		print("NN Inputs Shape: ", nn_inputs.shape)

		outputs = self.final_dataset["output"].values
		print("Outputs: ", outputs)

		input_ids = []
		attention_masks = []

		print('Start Tokenization')
		start_time = time.time()

		# For every review...
		for rev in inputs:
			# `encode_plus` will:
			#   (1) Tokenize the sentence.
			#   (2) Prepend the `[CLS]` token to the start.
			#   (3) Append the `[SEP]` token to the end.
			#   (4) Map tokens to their IDs.
			#   (5) Pad or truncate the sentence to `max_length`
			#   (6) Create attention masks for [PAD] tokens.
			encoded_dict = self.tokenizer.encode_plus(
				rev,  # Sentence to encode.
				add_special_tokens=True,  # Add '[CLS]' and '[SEP]'
				padding='max_length',
				max_length=self.tokenizer_max_length,  # Pad & truncate all sentences. Reviews over 512 so truncate there for now.
				pad_to_max_length=True,
				return_attention_mask=True,  # Construct attn. masks.
				return_tensors='pt',  # Return pytorch tensors.
				truncation=True  # Truncate at max_length
			)

			# Add the encoded sentence to the list.
			input_ids.append(encoded_dict['input_ids'])

			# And its attention mask (simply differentiates padding from non-padding).
			attention_masks.append(encoded_dict['attention_mask'])

		end_time = time.time()
		elapsed_time = end_time - start_time
		print(f"Tokenizer Encoding: {elapsed_time:.4f} seconds")

		# Convert the lists into tensors.
		input_ids = torch.cat(input_ids, dim=0)
		attention_masks = torch.cat(attention_masks, dim=0)
		outputs = torch.tensor(outputs).clone().detach()

		if tensor_save_path is not None:
			torch.save({"bert_inputs": input_ids, "nn_inputs": nn_inputs, "attention_masks": attention_masks, "outputs": outputs}, tensor_save_path)


		return input_ids, nn_inputs, attention_masks, outputs

	def _create_dataloaders(self):
		"""Creates dataloaders for training, validation, and testing."""
		# Combine the training inputs into a TensorDataset.
		print("Bert input: ", self.bert_inputs.shape)
		print("NN input: ", self.nn_inputs.shape)
		print("Attention Mask: ", self.attention_masks.shape)
		print("Outputs: ", self.outputs.shape)
		dataset = TensorDataset(self.bert_inputs, self.nn_inputs, self.attention_masks, self.outputs)

		# Create a 75-15-10 train-validation-test split.

		# Calculate the number of samples to include in each set.
		train_size = int(0.75 * len(dataset))
		val_size_b4_test = int(len(dataset) - train_size)

		# Divide the dataset by randomly selecting samples.
		train_dataset, val_dataset = random_split(dataset, [train_size, val_size_b4_test])

		val_size = int(0.15 * len(dataset))
		test_size = int(len(val_dataset) - val_size)
		val_dataset, test_dataset = random_split(val_dataset, [val_size, test_size])

		print('{:>5,} training samples'.format(train_size))
		print('{:>5,} validation samples'.format(val_size))
		print('{:>5,} testing samples'.format(test_size))

		# Create the DataLoaders for our training and validation sets.
		# We'll take training samples in random order.
		train_dataloader = DataLoader(
			train_dataset,  # The training samples.
			sampler=RandomSampler(train_dataset),  # Select batches randomly
			batch_size=self.batch_size  # Trains with this batch size.
		)

		# For validation the order doesn't matter, so we'll just read them sequentially.
		validation_dataloader = DataLoader(
			val_dataset,  # The validation samples.
			sampler=SequentialSampler(val_dataset),  # Pull out batches sequentially.
			batch_size=self.batch_size  # Evaluate with this batch size.
		)

		test_dataloader = DataLoader(
			test_dataset,  # The validation samples.
			sampler=SequentialSampler(test_dataset),  # Pull out batches sequentially.
			batch_size=self.batch_size  # Evaluate with this batch size.
		)

		return train_dataloader, validation_dataloader, test_dataloader

	def _get_scheduler(self):
		"""Creates the learning rate scheduler."""
		total_steps = len(self.train_dataloader) * self.epochs
		scheduler = get_linear_schedule_with_warmup(self.optimizer,
													num_warmup_steps=0,  # Default value in run_glue.py
													num_training_steps=total_steps)
		return scheduler


	def forward(self, input_ids, attention_mask, nn_input):
		bert_outputs = self.model(input_ids, attention_mask)
		pooled_output = bert_outputs.pooler_output

		nn_outputs = self.nn_layers(nn_input)
		final_input = torch.cat((pooled_output, nn_outputs), dim=1)
		final_output = self.final_layers(final_input)

		return final_output

	def train_model(self):
		self.train_losses = []
		self.val_losses = []
		self.train_accuracies = []
		self.val_accuracies = []
		"""Trains the model."""
		for e in range(self.epochs):
			print(f"\n===== Epoch {e + 1}/{self.epochs} =====")

			# training
			print("Training started ...")
			self._train_epoch(e)

			# validation testing
			print("Testing started ...")
			self._test_epoch(self.validation_dataloader, e)

	def _train_epoch(self, epoch):
		"""Trains the model for one epoch."""
		self.train()
		train_loss = 0


		# Number of iterations equal to total train dataset / batch size
		for step, batch in enumerate(self.train_dataloader):
			# Print progress in epoch
			print(f"Progress: {step + 1}/{len(self.train_dataloader)}", end='\r')
			# Parse iterator tensor dataset for important information
			input_ids, nn_input, attention_mask, labels = [b.to(self.device) for b in batch]

			# print("Loaded in batch", end='\r')

			output = self.forward(input_ids, attention_mask, nn_input)
			
			# print("Fed forward", end='\r')
			labels = labels.float().view(-1, 1)
			loss = self.criterion(output, labels)
			# print("Calculate loss", end='\r')
			loss.backward()
			self.optimizer.step()
			self.scheduler.step()

			# print("Back Prop", end='\r')


			# accumulate train loss
			train_loss += loss

		# print completed result
		print()
		print('Train Loss: %f' % (train_loss))
		self.train_losses.append(float(train_loss))
		return train_loss

	def test_model(self):
		"""Evaluates the model on the test set. Returns accuracy."""
		self.val_accuracies = []
		
		return self._test_epoch(self.test_dataloader, self.epochs - 1)

	def _test_epoch(self, iterator, epoch):
		"""Evaluates the model on the given dataloader."""
		self.eval()
		correct = 0
		total = 0

		with torch.no_grad():
			for step, batch in enumerate(iterator):
				# Print progress in epoch
				print(f"Progress: {step + 1}/{len(iterator)}", end='\r')
				# Parse iterator tensor dataset for important information
				input_ids, nn_input, attention_mask, labels = [b.to(self.device) for b in batch]

				# generate prediction
				output = self.forward(input_ids, attention_mask, nn_input)
				
				prob = output.sigmoid()  # BCEWithLogitsLoss has sigmoid

				# print("Prob: ", prob)
				# print("Prob Shape: ", prob.shape)

				# record processed data count
				total += labels.size(0)

				# take the index of the highest prob as prediction output
				prediction = prob.detach().clone()
				print("Original prediction: ", prediction)
				THRESHOLD = 0.5
				prediction[prediction > THRESHOLD] = 1
				prediction[prediction <= THRESHOLD] = 0
				# print("Prediction: ", prediction)
				# print("Prediction Shape: ", prediction.shape)
				labels = labels.float().view(-1, 1)
				print("Prediction and Labels: ", torch.cat((prediction, labels), dim=1))
				# print("Labels: ", labels)
				# print("Labels Shape: ", labels.shape)
				# print("Correct: ", prediction.eq(labels).sum().item())
				# print("Incorrect: ", prediction.ne(labels).sum().item())
				
				correct += prediction.eq(labels).sum().item()

				# print()

		# print completed result
		acc = 100. * correct / total
		print('Correct: %i   / Total: %i / Test Accuracy: %f' % (correct, total, acc))
		self.val_accuracies.append(float(acc))
		return acc
	
	def predict(self, text, nn_input):
		"""Predicts the class of the input text."""
		self.eval()
		self.model.eval()

		# Tokenize the input text
		inputs = self.tokenizer(text, return_tensors="pt", max_length=self.tokenizer_max_length, padding="max_length", truncation=True)
		input_ids = inputs["input_ids"].to(self.device)
		attention_mask = inputs["attention_mask"].to(self.device)

		nn_input = torch.tensor(nn_input, dtype=torch.float32).to(self.device)

		with torch.no_grad():
			output = self.forward(input_ids, attention_mask, nn_input)
			prob = output.sigmoid()
			
		return prob.cpu().numpy()

	def save(self, save_path):
		"""Saves the trained model."""
		count = 0
		while os.path.exists(save_path + str(count)):
			count += 1

		self.model.save_pretrained(save_path + str(count))

		print(f"Model saved to {save_path + str(count)}")
	
		# Saving the training stuff
		data = {
			"train_loss": self.train_losses,
			"vall_accuracy": self.val_accuracies,
			"epochs": int(self.epochs),
		}

		# Define the file path where you want to save the JSON file
		file_path = os.path.join('models', f"training_data_{count}.json")

		# Save the dictionary as a JSON file
		with open(file_path, "w") as json_file:
			json.dump(data, json_file, indent=4)

		print(f"Training data saved to {file_path}")

		return save_path + str(count)
	
	def load(self, load_path):
		"""Loads a trained model."""
		self.model = BertModel.from_pretrained(load_path).to(self.device)

		return self



In [23]:

def create_tensor():
    data_path = 'data/lemm_stop_reviews_parquet'
    tensor_path = None
    # Can add tensor_path here to load in tensors from a file
    epochs=1
    n_part=1
    classifier = BertClassifier(data_path, tensor_path=tensor_path, n_part=n_part, epochs=epochs)
    # classifier.train_model()
    # model_name = classifier.save(os.path.join('models', 'berthybridtrained'))

    # send_email("✅ BERT Training complete and model saved", f"Model took {get_time_from_start(start_time)}\nEpochs: {epochs}\nNumber of parts: {n_part}\nModel name: {model_name}")

In [24]:

def train_model():
    data_path = 'data/lemm_stop_reviews_parquet'
    # tensor_path = 'data/tensors/tensors.pth'
    # Can add tensor_path here to load in tensors from a file
    # For the main training, using epochs=500
    epochs=1
    n_part=1
    classifier = BertClassifier(data_path, tensor_path=None, n_part=n_part, epochs=epochs)
    classifier.train_model()
    model_name = classifier.save(os.path.join('models', 'berthybridtrained'))

    send_email("✅ BERT Training complete and model saved", f"Model took {get_time_from_start(start_time)}\nEpochs: {epochs}\nNumber of parts: {n_part}\nModel name: {model_name}")

In [25]:

def test_model():
    data_path = 'data/lemm_stop_reviews_parquet'
    tensor_path = 'data/tensors/tensors_parts1.pth'
    model_name = 'berthybridtrained5'
    # Can add tensor_path here to load in tensors from a file
    epochs=1
    n_part=1
    classifier = BertClassifier(data_path, tensor_path=tensor_path, n_part=n_part, epochs=epochs)
    classifier = classifier.load(os.path.join('models', model_name))
    print(f"Accuracy: {classifier.test_model()}")
    


In [26]:
# create_tensor()

In [27]:
try:
    # train_model()
    test_model()
except Exception as e:
    traceback_string = "".join(traceback.format_exception(type(e), value=e, tb=sys.exc_info()[2]))
    send_email("❌ BERT Training failed", f"Model took {get_time_from_start(start_time)}\nError: {e}\nError stack trace: {traceback_string}")
    print(f"Error: {e}\nError stack trace: {traceback_string}")

Bert input:  torch.Size([324, 512])
NN input:  torch.Size([324, 7])
Attention Mask:  torch.Size([324, 512])
Outputs:  torch.Size([324])
  243 training samples
   48 validation samples
   33 testing samples
Original prediction:  tensor([[0.4750],
        [0.4533],
        [0.5695],
        [0.4691],
        [0.4940],
        [0.5113],
        [0.4711],
        [0.4770],
        [0.5027],
        [0.4955],
        [0.5196],
        [0.4699],
        [0.5087],
        [0.5250],
        [0.4774],
        [0.4795],
        [0.4808],
        [0.4767],
        [0.4798],
        [0.4897],
        [0.4476],
        [0.5208],
        [0.4863],
        [0.4490],
        [0.4738],
        [0.6003],
        [0.4860],
        [0.4858],
        [0.4798],
        [0.4895],
        [0.4602],
        [0.4907]], device='cuda:0')
Prediction and Labels:  tensor([[0., 0.],
        [0., 0.],
        [1., 1.],
        [0., 0.],
        [0., 1.],
        [1., 0.],
        [0., 0.],
        [0., 1.],
        [1